# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
part_one_df = pd.read_csv("Output/WeatherDataCities2.csv", encoding="utf-8")
part_one_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cidreira,-30.18,-50.21,62.58,91,0,16.80,BR,1599096251
1,Dzilam González,21.28,-88.93,79.97,86,99,8.10,MX,1599096251
2,Albany,42.60,-73.97,73.99,90,100,6.26,US,1599096114
3,Butaritari,3.07,172.79,83.39,73,100,13.13,KI,1599096251
4,Castro,-24.79,-50.01,57.85,96,7,5.95,BR,1599095993


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
humidity = part_one_df["Humidity"].astype(float)
max_humidity = humidity.max()
cities = part_one_df[["Lat", "Lng"]]

In [6]:
gmaps.configure(api_key=g_key) 
fig1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig1.add_layer(heat_layer)
fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
fewer_cities_df = part_one_df.loc[(part_one_df["Max Temp"] > 70) & (part_one_df["Max Temp"] < 80) & (part_one_df["Cloudiness"] == 0) & (part_one_df["Wind Speed"] < 10), :]
fewer_cities_df = fewer_cities_df.dropna(how='any')
fewer_cities_df.reset_index(inplace=True)
del fewer_cities_df['index']
fewer_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lázaro Cárdenas,17.96,-102.20,79.74,81,0,5.75,MX,1599096259
1,São Félix do Xingu,-6.64,-51.99,76.71,39,0,1.50,BR,1599096260
2,Nouadhibou,20.93,-17.03,77.00,61,0,9.17,MR,1599096266
3,Zhangye,38.93,100.45,70.09,22,0,1.23,CN,1599096267
4,Abu Samrah,35.30,37.18,75.45,68,0,1.36,SY,1599096274


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotellist = []

for i in range(len(fewer_cities_df)):
    lat = fewer_cities_df.loc[i]['Lat']
    lng = fewer_cities_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    try:
        hotellist.append(response['results'][0]['name'])
    except:
        hotellist.append("")
fewer_cities_df["Hotel Name"] = hotellist
fewer_cities_df = fewer_cities_df.dropna(how='any')
fewer_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Lázaro Cárdenas,17.96,-102.20,79.74,81,0,5.75,MX,1599096259,Lázaro Cárdenas
1,São Félix do Xingu,-6.64,-51.99,76.71,39,0,1.50,BR,1599096260,São Félix do Xingu
2,Nouadhibou,20.93,-17.03,77.00,61,0,9.17,MR,1599096266,Nouadhibou
3,Zhangye,38.93,100.45,70.09,22,0,1.23,CN,1599096267,Zhangye
4,Abu Samrah,35.30,37.18,75.45,68,0,1.36,SY,1599096274,Suruj


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fewer_cities_df.iterrows()]
locations = fewer_cities_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig1.add_layer(markers)


# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))

In [ ]:
fig1